In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# ENABLE IF USING COLAB
USE_COLAB = True

if USE_COLAB:
    import os
    from google.colab import drive
    
    FOLDERNAME = 'UMass_Amherst/NN_AMI/NN_Project'
    drive.mount('/content/drive')
    %cd /content/drive/My\ Drive/$FOLDERNAME

    if os.path.exists('./spectroconv/'):
      %cd ./spectroconv
      !pip install hub
      !pip install hub[audio]
      !pip install wandb
    else:
      print("Error, must first pull project repository from github to Drive using ColabGitCommands.ipynb")

In [ ]:
import torch
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from models import wb_experiment

In [ ]:
n_mels = 128
wb_config = {
    'project': 'spectroconv-debug',
    'entity': 'jakeval-colab',
    'group': 'lrlc' # use this only when training/evaluating/etc a CNN
}

config = {
    'data': {
        'train': {
            'name': f'nsynth-full-{n_mels}-train'
        },
        'val': {
            'name': f'nsynth-full-{n_mels}-val'
        }
    },
    'model': {
        'id': wb_experiment.ModelType.LRLC,
        'name': 'LRLC'
    },
    'logging': {
        'batch_log_interval': 250,
        'eval_every_batch': True,
        'eval_every_epoch': True,
    },
    'parameters': {
      # need:
      # local_dim 
      # rank
      # lcw 
      # lrlc_channels # 8
      'epochs': 3,
      'batch_size': 100,
      'optimizer': 'adam',
      'learning_rate': 0.0005,
      'dropout': 0.5, # probability of an element to be zeroed (0 = no dropout)
      'l2_reg': 0,
      'kernel_size': 5,
      'num_channels': 16,
      'num_conv_layers': 1,
      'linear_layer_size': 64,
    },
}

In [ ]:
trainer = wb_experiment.TrainExperiment(wb_config=wb_config, login_key=True)
trainer.run_train(config, save_model=False)